In [10]:
import tensorflow as tf
import numpy as np

In [2]:
y_true = tf.constant([[0., 1.], [0., 0.]])
y_pred = tf.constant([[1., 1.], [1., 0.]])

2022-09-09 20:38:32.737227: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
loss_val = (tf.math.log(y_true + 1) - tf.math.log(y_pred + 1)) ** 2
print(tf.reduce_sum(loss_val) / 4)

tf.Tensor(0.2402265, shape=(), dtype=float32)


In [20]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()
msle(y_true, y_pred).numpy()

0.24022643

In [30]:
a_loss = loss_val.numpy()[0,0]
a_loss = 0.2699

In [31]:
diff = np.e ** np.sqrt(a_loss)

In [32]:
diff

1.6812188029243489

In [39]:
msle([10000.], [11000.]).numpy()

0.009082219